In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
from geopandas import GeoDataFrame

In [2]:
path = r'G:\projects\UtilityDistricts\eweb\DrinkingWater\IllegalCampCoordination\Recieved\IllegalCampNotification_pro\MyProject4.gdb'

In [3]:
campsites = gpd.read_file(path, layer='illegal_camp_sites')

In [4]:
campsites.head(3)

,status,comments,date,submitted_by,dogs_present,unruly_inhabitants,hazardous_materials_present,biohazards_present,size_of_encampment,global_id,geometry
0,Abandoned,Contcated group 5/18 and gave notice.,2016-05-18T15:36:59+00:00,Youngblood,No,No,None,None,Small,ab8037ce-196f-4dcd-8f50-432ad3eab07f,POINT (648676.895 837520.043)
1,Cleaned-up,Cleaned up 5/18.,2016-05-18T15:38:19+00:00,Youngblood,No,No,No,No,Small,85eaca87-90a3-4255-a42c-f0b41f8f5c5b,POINT (648704.972 837641.883)
2,Occupied,Notified campers 5/18.,2016-05-18T15:39:57+00:00,Youngblood,No,None,None,None,Small,03ed666e-4ad3-4710-9a39-0995f1deeea2,POINT (648423.293 837925.831)


In [5]:
river_buffer = gpd.read_file(path, layer = 'LCHydro24K_polyselectionsele')

In [6]:
campsites = campsites.to_crs(epsg=2914)

In [7]:
campsites.shape

(753, 11)

In [8]:
campsites.loc[:, 'year'] = campsites.date.apply(lambda x: x.split('-')[0] if x is not None else x)

In [9]:
def get_pip(points, polygons):
    id_list = list(polygons.index)
    df = pd.DataFrame().reindex_like(points).dropna()
    for ID in id_list:
        pol = (polygons.loc[polygons.index==ID])
        pol.reset_index(drop = True, inplace = True)
        pip_mask = points.within(pol.loc[0, 'geometry'])
        pip_data = points.loc[pip_mask].copy()
        df = df.append(pip_data)
    df.reset_index(inplace=True, drop=True)
    return df 

In [10]:
within_buffer = get_pip(campsites, river_buffer)

In [11]:
within_buffer_gdf = GeoDataFrame(within_buffer, crs='EPSG:2914', geometry=within_buffer['geometry'])

In [12]:
within_buffer.shape

(185, 12)

In [13]:
def close_points(points, dist_threshold):
    dist = points.geometry.apply(lambda g: points.distance(g))
    dist_triu = dist.where(~np.tril(np.ones(dist.shape)).astype(bool))
    dist_triu = dist_triu.stack()
    sel_dist = dist_triu[dist_triu < dist_threshold]
    idx = []
    if sel_dist.shape[0] > 0:
        pairs = list(sel_dist.index)
        df = pd.DataFrame()
        for pair in pairs:
            inds = list(pair)
            idx = idx + inds
            sel_pts = points[points.index.isin(inds)]
            x = sel_pts.geometry.apply(lambda p: p.x).mean()
            y = sel_pts.geometry.apply(lambda p: p.y).mean()
            ndf = pd.DataFrame({'x':[x], 'y':[y]})
            df = df.append(ndf)
        pt_avg = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.x, df.y, crs='EPSG:2914'))
        pt_avg.loc[:, 'year'] = points.year.values[0]
        res = pt_avg[['year', 'geometry']]
        return res, idx
    else:
        print(f'There are not close points within {dist_threshold} feet in {points.year.values[0]}')
        return 0

In [14]:
def reorganize_points(points, dist_threshold):
    check_near_points = close_points(points, dist_threshold)
    if check_near_points == 0:
        pnts = points[['year', 'geometry']]
    else:
        avg_pnts = check_near_points[0]
        idx = check_near_points[1]
        other_pnts = points[~points.index.isin(idx)]
        pnts = other_pnts[['year', 'geometry']].append(avg_pnts, ignore_index=True)
    return pnts

In [15]:
years = list(map(lambda x: str(x), range(2019, 2023)))

In [16]:
outpath = r'G:\projects\UtilityDistricts\eweb\DrinkingWater\IllegalCampCoordination\Recieved\Summary Statistics\output'

In [17]:
within_buffer_gdf.year.unique()

array(['2016', '2017', '2018', '2019', '2020', '2021', None, '2022',
       '2023', '2012'], dtype=object)

In [18]:
years

['2019', '2020', '2021', '2022']

In [19]:
lst = []
for year in years:
    site_yr = reorganize_points(campsites[campsites.year == year], 50)
    site_buffer_yr = reorganize_points(within_buffer_gdf[within_buffer_gdf.year == year][['year', 'geometry']], 50)
    site_yr.to_file(outpath+f'\\campsites_{year}.shp')
    site_buffer_yr.to_file(outpath+f'\\campsites_within_buffer_{year}.shp')
    lst.append([year, site_yr.shape[0], site_buffer_yr.shape[0]])

There are not close points within 50 feet in 2020
There are not close points within 50 feet in 2020
There are not close points within 50 feet in 2022


In [20]:
df=pd.DataFrame(lst, columns=['Year', 'Total_Count', 'Count_within_Buffer'])

In [22]:
df.to_csv(outpath+'\\summary_stat.csv', index=False)